In [10]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
import re
from bs4 import BeautifulSoup
from io import StringIO
import pandas as pd
import requests
from datetime import datetime
import time

In [2]:
url = "https://www.transfermarkt.com/joshua-kimmich/marktwertverlauf/spieler/161056"

In [3]:
path_to_extension = r'C:\Users\Mirko\AppData\Local\Google\Chrome\User Data\Default\Extensions\cjpalhdlnbpafiamejdnhcphjbkeiagm\1.61.0_0'

chrome_options = Options()
chrome_options.add_argument('load-extension=' + path_to_extension)


driver = webdriver.Chrome(options=chrome_options)

In [9]:
driver.get(url)

In [ ]:
iframe = driver.find_element(By.ID, "sp_message_iframe_953358")
driver.switch_to.frame(iframe)
driver.find_element(By.XPATH, "/html/body/div/div[2]/div[3]/div[1]/div/button").click()
driver.switch_to.parent_frame()

In [328]:
result_list = []

In [329]:
values_raw = driver.find_element(By.CLASS_NAME,"box").find_element(By.CLASS_NAME, "chart-dots").find_elements(By.CSS_SELECTOR, "*")

In [ ]:
values_raw

In [ ]:
result = driver.find_element(By.CLASS_NAME, "chart-tooltip.svelte-xxbpuy").find_element(By.CLASS_NAME, "left.svelte-xxbpuy").text

In [334]:
result

'Jan 24, 2014\nMarket value: €500k\nClub: RB Leipzig\nAge: 18'

In [335]:
date = re.search("^.*\n", result).group(0)
date = re.sub("\n", "", date)	

value = re.search("Market value:.*\n", result).group(0)
value = re.sub("\n", "", value)	
value = re.sub(".*: ","", value)
value = re.sub("k","000", value)
value = re.search("\\d+", value).group(0)

club = re.search("Club:.*\n", result).group(0)
club = re.sub("\n", "", club)	
club = re.sub(".*: ","", club)

age = re.search("Age:.*", result).group(0)
age = re.sub("\n", "", age)	
age = re.sub(".*: ","", age)
age

'18'

In [ ]:
result_list.append({"Date": date, "Value": value, "Club": club, "Age": age})

In [337]:
result_list

[{'Date': 'Jan 24, 2014',
  'Value': '500000',
  'Club': 'RB Leipzig',
  'Age': '18'}]

In [12]:
basicurl = []
for year in reversed(range(2017,2025)):
    basicurl.append(f"https://www.transfermarkt.com/bundesliga/startseite/wettbewerb/L1/plus/?saison_id={year}")
    basicurl.append(f"https://www.transfermarkt.com/premier-league/startseite/wettbewerb/GB1/plus/?saison_id={year}")
    basicurl.append(f"https://www.transfermarkt.com/laliga/startseite/wettbewerb/ES1/plus/?saison_id={year}")
    basicurl.append(f"https://www.transfermarkt.com/ligue-1/startseite/wettbewerb/FR1/plus/?saison_id={year}")
    basicurl.append(f"https://www.transfermarkt.com/serie-a/startseite/wettbewerb/IT1/plus/?saison_id={year}")
 
basicurl

['https://www.transfermarkt.com/bundesliga/startseite/wettbewerb/L1/plus/?saison_id=2024',
 'https://www.transfermarkt.com/premier-league/startseite/wettbewerb/GB1/plus/?saison_id=2024',
 'https://www.transfermarkt.com/laliga/startseite/wettbewerb/ES1/plus/?saison_id=2024',
 'https://www.transfermarkt.com/ligue-1/startseite/wettbewerb/FR1/plus/?saison_id=2024',
 'https://www.transfermarkt.com/serie-a/startseite/wettbewerb/IT1/plus/?saison_id=2024',
 'https://www.transfermarkt.com/bundesliga/startseite/wettbewerb/L1/plus/?saison_id=2023',
 'https://www.transfermarkt.com/premier-league/startseite/wettbewerb/GB1/plus/?saison_id=2023',
 'https://www.transfermarkt.com/laliga/startseite/wettbewerb/ES1/plus/?saison_id=2023',
 'https://www.transfermarkt.com/ligue-1/startseite/wettbewerb/FR1/plus/?saison_id=2023',
 'https://www.transfermarkt.com/serie-a/startseite/wettbewerb/IT1/plus/?saison_id=2023',
 'https://www.transfermarkt.com/bundesliga/startseite/wettbewerb/L1/plus/?saison_id=2022',
 'h

In [130]:
def startdriver():
    path_to_extension = r'C:\Users\Mirko\AppData\Local\Google\Chrome\User Data\Default\Extensions\cjpalhdlnbpafiamejdnhcphjbkeiagm\1.61.0_0'
    chrome_options = Options()
    chrome_options.add_argument('load-extension=' + path_to_extension)
    driver = webdriver.Chrome(options=chrome_options)
    return driver

In [ ]:
final_table =pd.DataFrame()

for url in basicurl:
    time.sleep(0.2)
    driver.get(url)
    if (url == basicurl[0]):
        time.sleep(3)
        iframe = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "sp_message_iframe_953358")))
        driver.switch_to.frame(iframe)
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, "/html/body/div/div[2]/div[3]/div[1]/div/button"))).click()
        driver.switch_to.parent_frame()

    else:
        time.sleep(2)
    print(f"Getting data from {url}")

    html = driver.page_source
    print("proceed with beautifulsoup..")
    soup = BeautifulSoup(html, "html")
    html_table = soup.find(id = "yw1").find("table")
    table = pd.read_html(StringIO(str(html_table)))[0]
    club_name = table.loc[table["Club.1"].notnull(),"Club.1"]
    links_raw = soup.find(id = "yw1").find("tbody").find_all(class_ = "zentriert no-border-rechts")
    links = []
    for i in links_raw:
        links.append(f"https://www.transfermarkt.com{i.find("a")["href"]}")
    table = pd.DataFrame({"Club": club_name, "Link": links, "Season": f"{year}/{year+1}"})
    final_table = pd.concat([final_table, table], ignore_index = True)
    


Getting data from https://www.transfermarkt.com/bundesliga/startseite/wettbewerb/L1/plus/?saison_id=2024
proceed with beautifulsoup..
Getting data from https://www.transfermarkt.com/premier-league/startseite/wettbewerb/GB1/plus/?saison_id=2024
No cookies popup, proceed...
proceed with beautifulsoup..
Getting data from https://www.transfermarkt.com/laliga/startseite/wettbewerb/ES1/plus/?saison_id=2024
No cookies popup, proceed...
proceed with beautifulsoup..
Getting data from https://www.transfermarkt.com/ligue-1/startseite/wettbewerb/FR1/plus/?saison_id=2024
No cookies popup, proceed...
proceed with beautifulsoup..
Getting data from https://www.transfermarkt.com/serie-a/startseite/wettbewerb/IT1/plus/?saison_id=2024
No cookies popup, proceed...
proceed with beautifulsoup..
Getting data from https://www.transfermarkt.com/bundesliga/startseite/wettbewerb/L1/plus/?saison_id=2023
No cookies popup, proceed...
proceed with beautifulsoup..
Getting data from https://www.transfermarkt.com/premi

In [277]:
driver = startdriver()
time.sleep(5)
player_links = []
clublist = final_table["Link"].tolist()
clubdf = pd.DataFrame()
for i in clublist:
    print(f"Getting data from {i}")
    driver.get(i)
    if (i == clublist[0]):
        time.sleep(2)
        iframe = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "sp_message_iframe_953358")))
        driver.switch_to.frame(iframe)
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, "/html/body/div/div[2]/div[3]/div[1]/div/button"))).click()
        driver.switch_to.parent_frame()
    else:
    	time.sleep(2)
    html = driver.page_source
    soup = BeautifulSoup(html, "html")
    
    player_links_raw = soup.find(id = "yw1").find("tbody").find_all(class_ = "rechts hauptlink")
    for j in player_links_raw:
        zelle = j.find("a")
        if zelle is not None:
            player_links.append(f"https://www.transfermarkt.com{zelle["href"]}")
            
    season = re.search("id\\/.*$",clublist[0]).group(0)
    season = re.sub("id\\/","",season)
    season = f"{season}/{int(season)+1}"
    clublink = i
    result_df = pd.DataFrame({"club_link": clublink, "player_link": player_links, "season": season})
    clubdf = pd.concat([clubdf, result_df], ignore_index = True)

Getting data from https://www.transfermarkt.com/fc-bayern-munchen/startseite/verein/27/saison_id/2024
Getting data from https://www.transfermarkt.com/bayer-04-leverkusen/startseite/verein/15/saison_id/2024
Getting data from https://www.transfermarkt.com/rasenballsport-leipzig/startseite/verein/23826/saison_id/2024
Getting data from https://www.transfermarkt.com/borussia-dortmund/startseite/verein/16/saison_id/2024
Getting data from https://www.transfermarkt.com/vfb-stuttgart/startseite/verein/79/saison_id/2024
Getting data from https://www.transfermarkt.com/eintracht-frankfurt/startseite/verein/24/saison_id/2024
Getting data from https://www.transfermarkt.com/vfl-wolfsburg/startseite/verein/82/saison_id/2024
Getting data from https://www.transfermarkt.com/sc-freiburg/startseite/verein/60/saison_id/2024
Getting data from https://www.transfermarkt.com/tsg-1899-hoffenheim/startseite/verein/533/saison_id/2024
Getting data from https://www.transfermarkt.com/borussia-monchengladbach/startsei

In [564]:
def get_player_values(driver = driver, url = url):
    finished = False
    tries = 0
    while (finished == False & tries <= 50):
        try:
            #values_raw = driver.find_element(By.,"box").find_element(By.CLASS_NAME, "chart-dots").find_elements(By.CSS_SELECTOR, "*")
            values_raw = WebDriverWait(driver, 180).until(EC.presence_of_element_located((By.CLASS_NAME, 'box')))
            values_raw = WebDriverWait(values_raw, 180).until(EC.presence_of_element_located((By.CLASS_NAME, 'chart-dots')))
            values_raw = values_raw.find_elements(By.CSS_SELECTOR, '*')
            
            finished = True
            time.sleep(2)
        except:
            driver.close()
            driver = startdriver()
            time.sleep(2)
            driver.get(url)
            time.sleep(3)
            try:
                iframe = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "sp_message_iframe_953358")))
                driver.switch_to.frame(iframe)
                WebDriverWait(driver, 1).until(EC.presence_of_element_located((By.XPATH, "/html/body/div/div[2]/div[3]/div[1]/div/button"))).click()
                driver.switch_to.parent_frame()
            except:	
                print("No cookies popup, proceed...")
            finished = False
            tries += 1
            if (tries < 50):
                print(f"try # {tries} did noot work. i will try again. stop after 10 tries")
            else:
                print("i tried 50 times. i will stop now")
                break
    #print("-detected the value coordinates")
    result_list = []
    plot_field = driver.find_element(By.CSS_SELECTOR, '#tm-main > div.row > div.large-8.columns > div > tm-market-value-development-graph-extended > div > div > svg > rect')
    max_x = plot_field.location['x'] + plot_field.size['width']
    #print(f"max x = {max_x}")
   # print(f"max coo = {max_x_coo}")
    for i in values_raw:
        #print(".")
        #time.sleep(0.3)
        ActionChains(driver).move_to_element(i).perform()
       #print(f"current coo = ", i.location["x"])
        current_val = i.location['x'] + i.size['width']/2
        #print(f"current coo = {current_val}")
        if current_val > max_x:
            move_x = (max_x - current_val)
            #print(f"move x = {move_x}")
            ActionChains(driver).move_by_offset(move_x, 0).perform()
        #if i == values_raw[len(values_raw)-1]:
        #    ActionChains(driver).move_by_offset(-2, 0).perform() # a little tweak because the last element is not recognized	
        finished = False
        tries = 0
        while (finished == False & tries <= 10):
            try:
                #result = driver.find_element(By.CLASS_NAME, "box")
                #result = result.find_element(By.CLASS_NAME, "chart-container.svelte-gaklzv")
                #result = result.find_element(By.CLASS_NAME, "chart-tooltip.svelte-xxbpuy")
                #result = result.find_element(By.TAG_NAME, "p")
                result = WebDriverWait(driver, 180).until(EC.presence_of_element_located((By.CLASS_NAME, 'box')))
                result = WebDriverWait(result, 180).until(EC.presence_of_element_located((By.CLASS_NAME, 'chart-container.svelte-gaklzv')))
                result = WebDriverWait(result, 180).until(EC.presence_of_element_located((By.CLASS_NAME, 'chart-tooltip.svelte-xxbpuy')))
                result = WebDriverWait(result, 180).until(EC.presence_of_element_located((By.TAG_NAME, 'p')))
        
                finished = True
            except:
                driver.close()
                driver = startdriver()
                time.sleep(2)
                driver.get(url)
                time.sleep(3)
                try:
                    iframe = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "sp_message_iframe_953358")))
                    driver.switch_to.frame(iframe)
                    WebDriverWait(driver, 1).until(EC.presence_of_element_located((By.XPATH, "/html/body/div/div[2]/div[3]/div[1]/div/button"))).click()
                    driver.switch_to.parent_frame()
                except:	
                    print("No cookies popup, proceed...")
                finished = False
                tries += 1
                if (tries < 50):
                    print(f"try # {tries} did noot work. i will try again. stop after 10 tries")
                else:
                    print("i tried 50 times. i will stop now")
                    break
        result = result.text
        date = re.search("^.*\n", result).group(0)
        date = re.sub("\n", "", date)	
        
        value = re.search("Market value:.*\n", result).group(0)
        value = re.sub("\n", "", value)
        value = re.sub("Market value: ","", value)
        
        club = re.search("Club:.*\n", result).group(0)
        club = re.sub("\n", "", club)
        club = re.sub(".*: ","", club)
        
        age = re.search("Age:.*", result).group(0)
        age = re.sub("\n", "", age)
        age = re.sub(".*: ","", age)
        
        result_list.append({"Date": date, "Value": value, "Club": club, "Age": age})
    print("values are collected")
    html = driver.page_source 
    soup = BeautifulSoup(html, "html")
    
    name = soup.find(class_ = 'data-header').find("h1").text
    name = re.search("\n.*$", name).group(0)
    name = re.sub("\n","", name)
    name.lstrip().rstrip()
    
    birthdate = soup.find(class_ = 'data-header__details').find('ul').find("li").text 
    birthdate = re.search("\n.*\n$", birthdate).group(0)
    birthdate = re.sub("(\n)|(\\(.*\\))","", birthdate)
    birthdate.lstrip().rstrip()
    
    result_df = pd.DataFrame(result_list)
    result_df["player_name"] = name
    result_df["player_birthdate"] = birthdate
    return result_df

In [595]:
player_links_unique = clubdf["player_link"].unique().tolist()
scraped = all_players["player_link"].unique().tolist()
player_links_unique2 = [x  for x in player_links_unique if x not in scraped]

In [596]:
len(player_links_unique2)

0

In [592]:
player_links_unique2

['https://www.transfermarkt.com/afonso-figueiredo/marktwertverlauf/spieler/237000',
 'https://www.transfermarkt.com/anthony-ribelin/marktwertverlauf/spieler/334006',
 'https://www.transfermarkt.com/sebastien-salles-lamonge/marktwertverlauf/spieler/288251',
 'https://www.transfermarkt.com/morgan-amalfitano/marktwertverlauf/spieler/39883',
 'https://www.transfermarkt.com/yoann-gourcuff/marktwertverlauf/spieler/18912',
 'https://www.transfermarkt.com/sabri-toufiqui/marktwertverlauf/spieler/264966',
 'https://www.transfermarkt.com/somalia/marktwertverlauf/spieler/148754',
 'https://www.transfermarkt.com/jordan-sebban/marktwertverlauf/spieler/495486',
 'https://www.transfermarkt.com/ola-toivonen/marktwertverlauf/spieler/36500',
 'https://www.transfermarkt.com/anthony-maisonnial/marktwertverlauf/spieler/324829',
 'https://www.transfermarkt.com/kevin-theophile-catherine/marktwertverlauf/spieler/60603',
 'https://www.transfermarkt.com/florentin-pogba/marktwertverlauf/spieler/140497',
 'https:/

In [585]:
driver = startdriver()

In [586]:
#driver.get('https://www.transfermarkt.com/nils-petersen/marktwertverlauf/spieler/42936')
#driver.get('https://www.transfermarkt.com/eduardo-quaresma/marktwertverlauf/spieler/504073')
#driver.get('https://www.transfermarkt.com/ivan-strinic/marktwertverlauf/spieler/39799')
driver.get('https://www.transfermarkt.com/yves-baraye/marktwertverlauf/spieler/193994')

In [587]:
values_raw = WebDriverWait(driver, 180).until(EC.presence_of_element_located((By.CLASS_NAME, 'box')))
values_raw = WebDriverWait(values_raw, 180).until(EC.presence_of_element_located((By.CLASS_NAME, 'chart-dots')))
values_raw = values_raw.find_elements(By.CSS_SELECTOR, '*')

In [588]:
ActionChains(driver).move_to_element(values_raw[0]).perform()

In [548]:

plot_field = driver.find_element(By.CSS_SELECTOR, '#tm-main > div.row > div.large-8.columns > div > tm-market-value-development-graph-extended > div > div > svg > rect')
max_x = plot_field.location['x'] + plot_field.size['width']
values_raw = WebDriverWait(driver, 180).until(EC.presence_of_element_located((By.CLASS_NAME, 'box')))
values_raw = WebDriverWait(values_raw, 180).until(EC.presence_of_element_located((By.CLASS_NAME, 'chart-dots')))
values_raw = values_raw.find_elements(By.CSS_SELECTOR, '*')
current_val = values_raw[-2].location['x'] + values_raw[-2].size['width']/2

In [549]:
print(current_val)
print(max_x)
move_x = (max_x - current_val)
move_x

993.0
992


-1.0

In [486]:
driver = startdriver()

In [593]:
#all_players = pd.DataFrame()
try: 
    driver.close()
except: 
    pass
driver = startdriver()
for i in player_links_unique2:
    time.sleep(2)
    driver.get(i)
    if (i == player_links_unique2[0]):
        time.sleep(3)
        try:
            iframe = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.ID, "sp_message_iframe_953358")))
            driver.switch_to.frame(iframe)
            WebDriverWait(driver, 1).until(EC.presence_of_element_located((By.XPATH, "/html/body/div/div[2]/div[3]/div[1]/div/button"))).click()
            driver.switch_to.parent_frame()
        except:	
            print("No cookies popup, proceed...")
    else:
        time.sleep(4)
	
    one_player = get_player_values(driver,url= i)
    player_id = re.sub(".*spieler/","",i)
    one_player["player_id"] = player_id
    one_player["player_link"] = i
    print(f"Got player: {one_player["player_name"].tolist()[0].strip()}")
    all_players = pd.concat([all_players, one_player], ignore_index = True)


values are collected
Got player: Afonso Figueiredo
values are collected
Got player: Anthony Ribelin
values are collected
Got player: Sébastien Salles-Lamonge
values are collected
Got player: Morgan Amalfitano
values are collected
Got player: Yoann Gourcuff
values are collected
Got player: Sabri Toufiqui
values are collected
Got player: Somália
values are collected
Got player: Jordan Sebban
values are collected
Got player: Ola Toivonen
values are collected
Got player: Anthony Maisonnial
values are collected
Got player: Kévin Théophile-Catherine
values are collected
Got player: Florentin Pogba
values are collected
Got player: Ben Karamoko
values are collected
Got player: Cheikh M'Bengue
values are collected
Got player: Alexandros Katranis
values are collected
Got player: Alexander Søderlund
values are collected
Got player: David Alcibiade
values are collected
Got player: Guillaume Gillet
values are collected
Got player: Jules Iloki
values are collected
Got player: Najib Gandi
values are 

In [597]:
all_players.to_csv("data/players_market_value.csv", index = False)

In [359]:
all_players

,Date,Value,Club,Age,player_name,player_birthdate,player_id,player_link
0,"Mar 17, 2005",€75k,FC Schalke 04 U19,18,Manuel Neuer,"Mar 27, 1986",17259,https://www.transfermarkt.com/manuel-neuer/mar...
1,"Sep 16, 2005",€150k,FC Schalke 04,19,Manuel Neuer,"Mar 27, 1986",17259,https://www.transfermarkt.com/manuel-neuer/mar...
2,"Aug 21, 2006",€350k,FC Schalke 04,20,Manuel Neuer,"Mar 27, 1986",17259,https://www.transfermarkt.com/manuel-neuer/mar...
3,"Jan 15, 2007",€1.50m,FC Schalke 04,20,Manuel Neuer,"Mar 27, 1986",17259,https://www.transfermarkt.com/manuel-neuer/mar...
4,"Jun 21, 2007",€3.00m,FC Schalke 04,21,Manuel Neuer,"Mar 27, 1986",17259,https://www.transfermarkt.com/manuel-neuer/mar...
...,...,...,...,...,...,...,...,...
59475,"Sep 26, 2022",€25k,Deportivo Alavés B,18,Selu Diallo,"Oct 1, 2003 ...",1054931,https://www.transfermarkt.com/selu-diallo/mark...
59476,"Dec 29, 2022",€50k,Deportivo Alavés B,19,Selu Diallo,"Oct 1, 2003 ...",1054931,https://www.transfermarkt.com/selu-diallo/mark...
59477,"Jun 30, 2023",€150k,Deportivo Alavés B,19,Selu Diallo,"Oct 1, 2003 ...",1054931,https://www.transfermarkt.com/selu-diallo/mark...
59478,"Dec 20, 2023",€200k,Deportivo Alavés B,20,Selu Diallo,"Oct 1, 2003 ...",1054931,https://www.transfermarkt.com/selu-diallo/mark...


In [228]:

#driver.get("https://www.transfermarkt.com/stefano-sorrentino/marktwertverlauf/spieler/21891")
#one_player = get_player_values(driver)
driver.get("https://www.transfermarkt.com/juanma-herzog/marktwertverlauf/spieler/1146029")

In [229]:
values_raw = WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.CLASS_NAME, 'box')))
values_raw = WebDriverWait(values_raw, 30).until(EC.presence_of_element_located((By.CLASS_NAME, 'chart-dots')))
values_raw = values_raw.find_elements(By.CSS_SELECTOR, '*')

In [230]:
values_raw

[<selenium.webdriver.remote.webelement.WebElement (session="3bed6a85f1689b73d2cdec0cb110b9e1", element="f.4C527F2BF0C78AE1821F4F78C30B90AC.d.7C0D689343B2A07051E39B943471EF54.e.120")>,
 <selenium.webdriver.remote.webelement.WebElement (session="3bed6a85f1689b73d2cdec0cb110b9e1", element="f.4C527F2BF0C78AE1821F4F78C30B90AC.d.7C0D689343B2A07051E39B943471EF54.e.121")>]

In [234]:
ActionChains(driver).move_to_element(values_raw[1]).perform()
ActionChains(driver).move_by_offset(-2, 0).perform()

In [237]:
get_player_values(driver)

,Date,Value,Club,Age,player_name,player_birthdate
0,"Feb 12, 2024",€1.00m,Las Palmas Atlético,19,Juanma Herzog,"May 13, 2004"
1,"Jun 7, 2024",€1.00m,Las Palmas Atlético,20,Juanma Herzog,"May 13, 2004"


In [ ]:
result = WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.CLASS_NAME, 'box')))
result = WebDriverWait(result, 30).until(EC.presence_of_element_located((By.CLASS_NAME, 'chart-container.svelte-gaklzv')))
result = WebDriverWait(result, 30).until(EC.presence_of_element_located((By.CLASS_NAME, 'chart-tooltip.svelte-xxbpuy')))
result = WebDriverWait(result, 30).until(EC.presence_of_element_located((By.TAG_NAME, 'p')))
result.text
                                                                                                                                          